In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import glob
import sklearn.preprocessing as sp
from sklearn.cluster import KMeans,AgglomerativeClustering
from clustergram import Clustergram
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap,BoundaryNorm
from matplotlib.colorbar import ColorbarBase
import matplotlib as mpl
import matplotlib.patches as mpatches
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib import font_manager as fm


pd.set_option('display.max_columns', None)
#set font globally
plt.rcParams['font.family'] = 'Arial'

In [ ]:
all_measure = pd.read_csv(os.getcwd()+'\\all_measure(40).csv',index_col = 0)

In [ ]:
#drop variable with too many missing values 
all_measure1 = all_measure.drop(['Renewable only'], axis = 1)
#normalised transformation and standardisation
all_measure_boxcox_std = pd.DataFrame(sp.PowerTransformer(method = 'yeo-johnson',standardize = True).fit_transform(all_measure1), \
                                      columns = all_measure1.columns, index = all_measure1.index)

* Feature selection

In [ ]:
# correlation matrix
corr = all_measure_boxcox_std.corr()
#corr.columns = list(range(1,all_measure_boxcox_std.shape[1]+1))
# Create a mask
mask = np.triu(np.ones_like(corr, dtype= bool))
mask = pd.DataFrame(mask.T,columns = corr.columns, index = corr.index)
corr = corr.mask(~mask)

In [ ]:
#drop variable with strong correlation
drop_var = ['Retired','Universal credit','Owns outright','Co2 emissions','Under occupancy','Prepay electricity']
dropped_data = all_measure_boxcox_std.drop(drop_var, axis = 1).fillna(0)

* Clustergram to determine optimal k


In [ ]:
def clustergram_plot(n,df,name):
    cgram = Clustergram(range(1, n), n_init=100,random_state = 0, algorithm = 'elkan')
    cgram.fit(df)

    ax = cgram.plot(
        figsize=(8, 6),
        size = 0.7,
        linewidth=0.6,
        line_style=dict(color='black'),
        cluster_style={"color": '#e41a1c'},
        pca_weighted=True
    )
    ax.yaxis.grid(False)
    return plt.show()

In [ ]:
clustergram_plot(11,dropped_data,'super_40variables')

* Kmeans clustering

In [ ]:
def kmeans(k,drop_data,raw_data):
    cluster = KMeans(n_clusters=k, random_state = 0, n_init = 100, max_iter = 10000).fit_predict(drop_data)
    #Assign the each cluster number to the merged data
    df = raw_data.assign(lbls = cluster)
    return df

In [ ]:
data = kmeans(6,dropped_data,all_measure)

In [ ]:
# read boundary data
geo_ew = gpd.read_file(os.getcwd()+'/boundaries/LSOA_(Dec_2021)_Boundaries_Generalised_Clipped_EW_(BGC).geojson').set_index('LSOA21CD')[['geometry']]#epsg 4326
geo_s = gpd.read_file(os.getcwd() + '/boundaries/SG_DataZone_Bdry_2011.shp').set_index('DataZone')[['geometry']]
geo_gb = pd.concat([geo_ew.to_crs(geo_s.crs),geo_s])


In [ ]:
#join dataframe with geometry
all_measure_geo = geo_gb.join(data)#4326
# index scores for Supergroups
all_measure_geo.iloc[:,1:-1] = ((all_measure_geo.iloc[:,1:-1]/all_measure_geo.iloc[:,1:-1].mean())*100).round()
index_score_super = all_measure_geo.groupby('lbls').mean().round().T


* Further division for finer Group level

In [ ]:
def standardise(df):
    output = pd.DataFrame(sp.PowerTransformer(method = 'yeo-johnson',standardize = True).fit_transform(df), columns = df.columns, index = df.index)
    return output

In [ ]:
# divide classification by Supergroups
lbls = [list(all_measure_geo.groupby('lbls'))[i][1] for i in range(6)]# or ['A','B','C','D','E']
variable_dropped = ['Renewable only','Retired','Co2 emissions','Prepay electricity','Under occupancy','Universal credit','Owns outright','geometry','lbls']

In [ ]:
group_A = lbls[0].drop(variable_dropped, axis = 1).fillna(0)
group_A = standardise(group_A)
clustergram_plot(5,group_A,'superA')

In [ ]:
lbls_A = kmeans(3, group_A, lbls[0])

In [ ]:
group_B = lbls[1].drop(variable_dropped, axis = 1).fillna(0)
group_B = standardise(group_B)
clustergram_plot(5,group_B,'superB')

In [ ]:
lbls_B = kmeans(2, group_B, lbls[1])

In [ ]:
group_C = lbls[2].drop(variable_dropped, axis = 1).fillna(0)
group_C = standardise(group_C)
clustergram_plot(5,group_C,'superC')

In [ ]:
lbls_C = kmeans(2, group_C, lbls[2])

In [ ]:
group_D = lbls[3].drop(variable_dropped, axis = 1).fillna(0)
group_D = standardise(group_D)
clustergram_plot(5,group_D,'superD')

In [ ]:
lbls_D = kmeans(2, group_D, lbls[3])

In [ ]:
group_E = lbls[4].drop(variable_dropped, axis = 1).fillna(0)
group_E = standardise(group_E)
clustergram_plot(5,group_E,'superE')

In [ ]:
lbls_E = kmeans(3, group_E, lbls[4])

In [ ]:
group_F = lbls[5].drop(variable_dropped, axis = 1).fillna(0)
group_F = standardise(group_F)
clustergram_plot(5,group_F,'superF')

In [ ]:
lbls_F = kmeans(2, group_F, lbls[5])

In [ ]:
lbls_ = [lbls_A,lbls_B,lbls_C,lbls_D,lbls_E,lbls_F]
#rename sub-category
sub_name = [['A1','A2','A3'],['B1','B2'],['C1','C2'],['D1','D2'], ['E1','E2','E3'],['F1','F2']]
# rename labels with 'A-1','A-2','B-1'...
for a in range(6):
    lbls_[a].loc[lbls_[a]['lbls'] == 0, 'lbls'] = sub_name[a][0]
    lbls_[a].loc[lbls_[a]['lbls'] == 1, 'lbls'] = sub_name[a][1]
lbls_[0].loc[lbls_[0]['lbls'] == 2, 'lbls'] = sub_name[0][2]
lbls_[4].loc[lbls_[4]['lbls'] == 2, 'lbls'] = sub_name[4][2]

In [ ]:
#14 Groups
super_sub = pd.concat(lbls_)

In [ ]:
def index_score(f):
    all_ = f.drop('geometry', axis  = 1)
    all_.iloc[:,:-1] = (all_.iloc[:,:-1]/all_.iloc[:,:-1].mean())*100
    #all_.rename(columns = {'Disabled or long sick':'Disabled/Sick'},inplace = True)
    index_score = all_.groupby('lbls').mean().round()
    return index_score

In [ ]:
# calculate index score
score_national = index_score(super_sub)
#write to csv
score_national.to_csv('index_score(Groups).csv')

In [ ]:
super_sub['supergroups'] = super_sub['lbls'].str[0]
super_sub.rename(columns={'lbls':'groups'},inplace=True)
super_sub.head()

In [ ]:
# write the whole data to one geopackage
super_sub.to_file('EDC_tier1&2_gb.shp', crs = super_sub.crs)